In [1]:
%run ../functions_scripts/database_functions.ipynb
%run ../functions_scripts/dataframe_cleaning_functions.ipynb

In [3]:

from pathlib import Path
from typing import List, Optional
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import text
from psycopg2.extras import execute_values
from sqlalchemy_utils import database_exists, create_database
import logging
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

RAW_DIR = Path(os.getenv("RAW_DIR"))
CLEAN_DIR = Path(os.getenv("CLEAN_DIR"))
ADMIN_DB_URL = os.getenv("ADMIN_DB_URL")
DB_URL = os.getenv("DB_URL")



In [4]:
script_silver_ddl = """
CREATE SCHEMA IF NOT EXISTS silver;

CREATE TABLE IF NOT EXISTS silver.caracteristiques (
  identifiant_de_l_accident VARCHAR(64),
  date_et_heure TIMESTAMP,
  jour SMALLINT,
  mois SMALLINT,
  annee SMALLINT,
  heure_minute TIME,
  date DATE,
  year_georef SMALLINT,
  lumiere VARCHAR(10),
  code_postal VARCHAR(10),
  code_insee VARCHAR(20),
  departement VARCHAR(6),
  commune VARCHAR(10),
  code_commune VARCHAR(10),
  code_officiel_commune VARCHAR(20),
  nom_officiel_commune VARCHAR(255),
  nom_officiel_commune_arrondissement_municipal VARCHAR(255),
  code_officiel_departement VARCHAR(10),
  nom_officiel_departement VARCHAR(255),
  code_officiel_region VARCHAR(10),
  nom_officiel_region VARCHAR(255),
  code_officiel_epci VARCHAR(20),
  nom_officiel_epci VARCHAR(255),
  localisation SMALLINT,
  intersection SMALLINT,
  conditions_atmospheriques SMALLINT,
  collision SMALLINT,
  adresse VARCHAR(512),
  gps CHAR(1),
  latitude DOUBLE PRECISION,
  longitude DOUBLE PRECISION,
  coordonnees VARCHAR(255),
  numero VARCHAR(64)
);

CREATE TABLE IF NOT EXISTS silver.lieux(
  identifiant_de_l_accident VARCHAR(64),
  categorie_route SMALLINT,
  voie VARCHAR(128),
  v1 VARCHAR(32),
  v2 VARCHAR(32),
  circulation SMALLINT,
  nombre_de_voies SMALLINT,
  voie_reservee SMALLINT,
  profil SMALLINT,
  pr VARCHAR(64),
  pr1 INT,
  plan SMALLINT,
  largeur_terre_plein_central DOUBLE PRECISION,
  largeur_de_la_chaussee DOUBLE PRECISION,
  surface SMALLINT,
  infrastructure SMALLINT,
  situation SMALLINT,
  env1 SMALLINT
);

CREATE TABLE IF NOT EXISTS silver.vehicules (
  identifiant_de_l_accident VARCHAR(64),
  identifiant_vehicule VARCHAR(64),
  sens SMALLINT,
  categorie_vehicule SMALLINT,
  obstacle_fixe_heurte SMALLINT,
  obstacle_mobile_heurte SMALLINT,
  point_de_choc SMALLINT,
  manoeuvre SMALLINT,
  nombre_d_occupants SMALLINT
);

CREATE TABLE IF NOT EXISTS silver.usagers (
  identifiant_de_l_accident VARCHAR(64),
  identifiant_vehicule VARCHAR(64),
  place VARCHAR(128),
  categorie_d_usager SMALLINT,
  gravite SMALLINT,
  sexe SMALLINT,
  annee_de_naissance SMALLINT,
  motif_trajet SMALLINT,
  existence_equipement_de_securite VARCHAR(8),
  utilisation_equipement_de_securite SMALLINT,
  localisation_du_pieton SMALLINT,
  action_pieton SMALLINT,
  pieton_seul_ou_non SMALLINT
);
"""
engine = get_engine(DB_URL) 
execute_script(engine, script_silver_ddl)




INFO:__main__:Exécution du script SQL...
INFO:__main__:Script SQL exécuté avec succès
INFO:__main__:Connexion fermée et engine libéré.


# 🧹 Nettoyage de la table `caracteristiques`


In [ ]:
query = "SELECT COUNT(*) FROM bronze.caracteristiques_raw"
count= get_single_value(engine, query)
print(f"Nombre de lignes dans bronze.caracteristiques_raw : {count}")

Nombre de lignes dans bronze.caracteristiques_raw : 475911


In [6]:
#afficher un echantillon de 20 lignes
df_caracteristiques = get_df_from_table(engine, "bronze", "caracteristiques_raw")
# df_caracteristiques_sample = df_caracteristiques.sample(100, random_state=42)
# df_caracteristiques_sample.head()
df_caracteristiques.head()


,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,date,year_georef,lumiere,code_postal,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201800018092,2018-06-17T17:40:00+02:00,17,06,2018,17:40,None,2018,Plein jour,80150,...,Hors agglomération,1,Normale,Sans collision,D928,Métropole,5019561,0188016,"50.186673, 1.879133",928
1,201600052725,2016-12-07T14:40:00+01:00,07,12,2016,14:40,None,2016,Plein jour,75009,...,En agglomération,2,Normale,Autre collision,"62, RUE DE LA CHAUSSEE D",Métropole,4887530,0233229,"48.875125, 2.332459",62
2,201800018104,2018-02-17T17:00:00+01:00,17,02,2018,17:00,2018-04-01,2018,Plein jour,80370,...,En agglomération,9,Normale,Autre collision,R DE LA VACQUERIE,Métropole,5012380,0216687,"50.128037, 2.161891",None
3,201600054636,2016-08-27T00:01:00+02:00,27,08,2016,00:01,None,2016,Nuit avec éclairage public allumé,75015,...,En agglomération,5,Normale,Deux véhicules – par le coté,"10, RUE DU LAOS",Métropole,4884990,0230110,"48.8501, 2.300974",10
4,201800019309,2018-10-07T19:30:00+02:00,07,10,2018,19:30,2018-09-01,2018,Nuit avec éclairage public allumé,57560,...,En agglomération,1,Normale,Autre collision,119 rue du Général Jordy,Métropole,4863570,0709861,"48.635693, 7.098653",119


In [8]:
# Supprimer les doublons
print(f"Nombre de lignes avant suppression des doublons : {df_caracteristiques.shape[0]}")
df_caracteristiques = df_caracteristiques.drop_duplicates()
print(f"Nombre de lignes après suppression des doublons : {df_caracteristiques.shape[0]}")

#remplacer les valeurs nulles
remplacer_valeurs_null(df_caracteristiques)

Nombre de lignes avant suppression des doublons : 475911
Nombre de lignes après suppression des doublons : 475911


,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,date,year_georef,lumiere,code_postal,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201800018092,2018-06-17T17:40:00+02:00,17,06,2018,17:40,Inconnu,2018,Plein jour,80150,...,Hors agglomération,1,Normale,Sans collision,D928,Métropole,5019561,0188016,"50.186673, 1.879133",928
1,201600052725,2016-12-07T14:40:00+01:00,07,12,2016,14:40,Inconnu,2016,Plein jour,75009,...,En agglomération,2,Normale,Autre collision,"62, RUE DE LA CHAUSSEE D",Métropole,4887530,0233229,"48.875125, 2.332459",62
2,201800018104,2018-02-17T17:00:00+01:00,17,02,2018,17:00,2018-04-01,2018,Plein jour,80370,...,En agglomération,9,Normale,Autre collision,R DE LA VACQUERIE,Métropole,5012380,0216687,"50.128037, 2.161891",Inconnu
3,201600054636,2016-08-27T00:01:00+02:00,27,08,2016,00:01,Inconnu,2016,Nuit avec éclairage public allumé,75015,...,En agglomération,5,Normale,Deux véhicules – par le coté,"10, RUE DU LAOS",Métropole,4884990,0230110,"48.8501, 2.300974",10
4,201800019309,2018-10-07T19:30:00+02:00,07,10,2018,19:30,2018-09-01,2018,Nuit avec éclairage public allumé,57560,...,En agglomération,1,Normale,Autre collision,119 rue du Général Jordy,Métropole,4863570,0709861,"48.635693, 7.098653",119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600058189,2016-02-18T19:08:00+01:00,18,02,2016,19:08,Inconnu,2016,Nuit avec éclairage public non allumé,97130,...,En agglomération,1,Normale,Autre collision,DELGRES ( BOULEVARD ),Antilles (Martinique ou Guadeloupe),Inconnu,Inconnu,"16.044166, -61.565258",Inconnu
475907,201800017928,2018-05-31T09:30:00+02:00,31,05,2018,09:30,Inconnu,2018,Plein jour,82300,...,Hors agglomération,2,Pluie légère,Trois véhicules et plus – en chaîne,RD964 intersection RD95,Métropole,4412296,0158603,"44.144565, 1.611812",964
475908,201600058951,2016-04-22T07:45:00+02:00,22,04,2016,07:45,Inconnu,2016,Plein jour,97420,...,En agglomération,1,Normale,Deux véhicules – par le coté,COMMUNE DE PARIS (AVENUE,Réunion,Inconnu,Inconnu,"-20.939385, 55.290913",Inconnu
475909,201800018085,2018-07-21T16:55:00+02:00,21,07,2018,16:55,Inconnu,2018,Plein jour,80130,...,Hors agglomération,1,Normale,Autre collision,D925,Métropole,5007348,0154298,"50.086456, 1.542519",925


In [130]:
#changer type de colonne date_et_heure

df_caracteristiques['date_et_heure'] =pd.to_datetime(

    df_caracteristiques['date_et_heure'],
    errors='raise',
    utc=True
    ).to_frame()
# df_caracteristiques_sample.drop(['annee', 'mois', 'jour', 'heure_minute'], axis=1, inplace=True) 
df_caracteristiques

,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,date,year_georef,lumiere,code_postal,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201700033619,2017-05-19 15:40:00+00:00,19,05,2017,17:40,None,2017,Plein jour,66100,...,Hors agglomération,6,Normale,Deux véhicules – par l’arrière,COPENHAGUE (ROND POINT D,Métropole,4269110,0285200,"42.668322, 2.897617",None
1,201700048262,2017-12-25 16:55:00+00:00,25,12,2017,17:55,None,2017,Crépuscule ou aube,93200,...,Hors agglomération,1,Normale,Trois véhicules et plus – en chaîne,AUTOROUTE A1,Métropole,None,None,None,1
2,201700048288,2017-01-01 16:25:00+00:00,01,01,2017,17:25,None,2017,Nuit sans éclairage public,92230,...,Hors agglomération,1,Temps couvert,Trois véhicules et plus – en chaîne,A86,None,None,None,"48.925197, 2.293085",86
3,201700048486,2017-05-03 22:04:00+00:00,04,05,2017,00:04,None,2017,Nuit avec éclairage public non allumé,78330,...,Hors agglomération,1,Autre,Autre collision,A12 Y,Métropole,4880700,0205200,"48.812878, 2.048119",12
4,201700048639,2017-07-11 06:10:00+00:00,11,07,2017,08:10,None,2017,Plein jour,78140,...,Hors agglomération,1,Normale,Deux véhicules – par le coté,N118,Métropole,4878430,0222310,"48.783326, 2.187486",118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600004642,2016-01-01 04:00:00+00:00,01,01,2016,05:00,None,2016,Nuit sans éclairage public,01290,...,En agglomération,1,Brouillard - fumée,Deux véhicules - frontale,Les Allouets,Métropole,None,None,"46.21663, 4.951376",None
475907,201600004875,2016-07-27 14:15:00+00:00,27,07,2016,16:15,None,2016,Plein jour,38530,...,En agglomération,9,Normale,Autre collision,parking av de la gare,Métropole,None,None,"45.433123, 6.015207",None
475908,201600000781,2016-01-14 07:00:00+00:00,14,01,2016,08:00,None,2016,Nuit avec éclairage public allumé,27400,...,En agglomération,2,Pluie forte,Deux véhicules - frontale,RD 164,Métropole,None,None,None,164
475909,201400047990,2014-03-08 16:15:00+00:00,08,03,2014,17:15,None,2015,Plein jour,92210,...,Hors agglomération,1,Normale,Trois véhicules et plus - collisions multiples,A13,None,None,None,"48.845569, 2.215131",13


In [9]:
#Harmonisation de la colonne 'intersection'
print(df_caracteristiques.groupby('intersection').size().reset_index(name='counts').sort_values(by='counts', ascending=False))
mapping_intersection = {
    '0': 'non renseigné',
    '1': 'hors intersection',
    '2': 'intersection en x',
    '3': 'intersection en t',
    '4': 'intersection en y',
    '5': 'intersection à plus de 4 branches',
    '6': 'giratoire',
    '7': 'place',
    '8': 'passage à niveau',
    '9': 'autre intersection'
}
harmonise_colonne_df(df_caracteristiques,'intersection',mapping_intersection)
print('\napres harmonisation:')
print(df_caracteristiques.groupby('intersection').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

  intersection  counts
1            1  328937
2            2   56882
3            3   44672
6            6   15386
9            9   10060
4            4    8361
7            7    6100
5            5    4833
8            8     590
0            0      90

apres harmonisation:
                        intersection  counts
2                  hors intersection  328937
4                  intersection en x   56882
3                  intersection en t   44672
1                          giratoire   15386
0                 autre intersection   10060
5                  intersection en y    8361
9                              place    6100
6  intersection à plus de 4 branches    4833
8                   passage à niveau     590
7                      non renseigné      90


In [132]:
# -1 
print(df_caracteristiques.groupby('collision').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

df_caracteristiques = remplacer_valeur(
    df_caracteristiques,
    'collision',
    '-1',
    'Inconnu'
)
print(df_caracteristiques.groupby('collision').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

                                        collision  counts
1                                 Autre collision  161298
3                    Deux véhicules – par le coté  132605
4                  Deux véhicules – par l’arrière   58352
5                                  Sans collision   46028
2                       Deux véhicules - frontale   45192
7             Trois véhicules et plus – en chaîne   16916
6  Trois véhicules et plus - collisions multiples   15508
0                                              -1       2
                                        collision  counts
0                                 Autre collision  161298
2                    Deux véhicules – par le coté  132605
3                  Deux véhicules – par l’arrière   58352
5                                  Sans collision   46028
1                       Deux véhicules - frontale   45192
7             Trois véhicules et plus – en chaîne   16916
6  Trois véhicules et plus - collisions multiples   15508
4             

In [133]:

#supprimer les colonnes avec missing values supérieures à 50%
# col_a_supprimer = ['infrastructure']

# df_caracteristiques = supprimer_colonnes(df_caracteristiques, col_a_supprimer)

df_caracteristiques

,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,date,year_georef,lumiere,code_postal,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201700033619,2017-05-19 15:40:00+00:00,19,05,2017,17:40,None,2017,Plein jour,66100,...,Hors agglomération,giratoire,Normale,Deux véhicules – par l’arrière,COPENHAGUE (ROND POINT D,Métropole,4269110,0285200,"42.668322, 2.897617",None
1,201700048262,2017-12-25 16:55:00+00:00,25,12,2017,17:55,None,2017,Crépuscule ou aube,93200,...,Hors agglomération,hors intersection,Normale,Trois véhicules et plus – en chaîne,AUTOROUTE A1,Métropole,None,None,None,1
2,201700048288,2017-01-01 16:25:00+00:00,01,01,2017,17:25,None,2017,Nuit sans éclairage public,92230,...,Hors agglomération,hors intersection,Temps couvert,Trois véhicules et plus – en chaîne,A86,None,None,None,"48.925197, 2.293085",86
3,201700048486,2017-05-03 22:04:00+00:00,04,05,2017,00:04,None,2017,Nuit avec éclairage public non allumé,78330,...,Hors agglomération,hors intersection,Autre,Autre collision,A12 Y,Métropole,4880700,0205200,"48.812878, 2.048119",12
4,201700048639,2017-07-11 06:10:00+00:00,11,07,2017,08:10,None,2017,Plein jour,78140,...,Hors agglomération,hors intersection,Normale,Deux véhicules – par le coté,N118,Métropole,4878430,0222310,"48.783326, 2.187486",118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600004642,2016-01-01 04:00:00+00:00,01,01,2016,05:00,None,2016,Nuit sans éclairage public,01290,...,En agglomération,hors intersection,Brouillard - fumée,Deux véhicules - frontale,Les Allouets,Métropole,None,None,"46.21663, 4.951376",None
475907,201600004875,2016-07-27 14:15:00+00:00,27,07,2016,16:15,None,2016,Plein jour,38530,...,En agglomération,autre intersection,Normale,Autre collision,parking av de la gare,Métropole,None,None,"45.433123, 6.015207",None
475908,201600000781,2016-01-14 07:00:00+00:00,14,01,2016,08:00,None,2016,Nuit avec éclairage public allumé,27400,...,En agglomération,intersection en x,Pluie forte,Deux véhicules - frontale,RD 164,Métropole,None,None,None,164
475909,201400047990,2014-03-08 16:15:00+00:00,08,03,2014,17:15,None,2015,Plein jour,92210,...,Hors agglomération,hors intersection,Normale,Trois véhicules et plus - collisions multiples,A13,None,None,None,"48.845569, 2.215131",13


# 🧹 Nettoyage de la table Véhicule 


In [134]:
df_vehicles = get_df_from_table(engine, "bronze", "vehicules_raw")
df_vehicles.head()

,identifiant_de_l_accident,identifiant_vehicule,sens,categorie_vehicule,obstacle_fixe_heurte,obstacle_mobile_heurte,point_de_choc,manoeuvre,nombre_d_occupants
0,201700033619,"B01,A01",None,"VL seul,VL seul",None,"Véhicule,Véhicule","Arrière,Avant",Sans changement de direction,None
1,201700048262,"A01,E01,C01,B01,D01",PK ou PR ou numéro d’adresse postale croissant...,"VL seul,VL seul,VL seul,VL seul,VL seul",None,"Véhicule,Véhicule,Véhicule,Véhicule,Véhicule","Avant droit,Arrière,Côté gauche,Arrière droit,...","Manœuvre d’évitement,Arrêté (hors stationnemen...",None
2,201700048288,"D01,B01,A01,C01",PK ou PR ou numéro d’adresse postale décroissa...,"VL seul,VL seul,VL seul,VL seul",None,"Véhicule,Véhicule,Véhicule,Véhicule","Arrière,Avant,Avant,Arrière","Même sens,Même sens,Même sens,Même sens",None
3,201700048486,"A01,B01",PK ou PR ou numéro d’adresse postale croissant...,"VL seul,VL seul",None,"Véhicule,Véhicule","Avant gauche,Côté gauche","Changeant de file A gauche,Déporté A gauche",None
4,201700048639,"Z01,A01",PK ou PR ou numéro d’adresse postale décroissa...,"VL seul,Motocyclette > 125 cm3",None,"Véhicule,Véhicule",Avant droit,Sans changement de direction,None


In [ ]:
#supprimer les colonnes avec missing values supérieures à 50%
col_a_supprimer = ['sens', 'obstacle_fixe_heurte', 'obstacle_mobile_heurte', 'nombre_d_occupants']
df_vehicles = supprimer_colonnes(df_vehicles, col_a_supprimer)

# 🧹 Nettoyage de la table usagers 


In [157]:
df_usagers = get_df_from_table(engine, "bronze", "usagers_raw")
df_usagers.head()

,identifiant_de_l_accident,identifiant_vehicule,place,categorie_d_usager,gravite,sexe,annee_de_naissance,motif_trajet,existence_equipement_de_securite,utilisation_equipement_de_securite,localisation_du_pieton,action_pieton,pieton_seul_ou_non
0,201700033619,"B01,A01","1,1,2,2","Conducteur,Conducteur,Passager,Passager","Blessé,Blessé,Blessé,Blessé","Masculin,Masculin,Masculin,Masculin","1996,1939,1996,1931","Promenade – loisirs,Promenade – loisirs,Promen...","Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
1,201700048262,"A01,E01,C01,B01,D01","1,1,1,1,1","Conducteur,Conducteur,Conducteur,Conducteur,Co...","Indemne,Indemne,Blessé,Indemne,Indemne","Masculin,Masculin,Féminin,Féminin,Masculin","1952,1934,1979,1986,1978",None,"Ceinture,Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
2,201700048288,"D01,B01,A01,C01","1,1,1,2,1","Conducteur,Conducteur,Conducteur,Passager,Cond...","Indemne,Indemne,Indemne,Blessé,Indemne","Masculin,Masculin,Masculin,Masculin,Masculin","1982,1978,1990,2004,1978",None,"Ceinture,Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
3,201700048486,"A01,B01","1,1,2","Conducteur,Conducteur,Passager","Blessé,Indemne,Blessé","Masculin,Masculin,Masculin","1992,1990,1992",Autre,"Ceinture,Ceinture,Ceinture","Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant",None
4,201700048639,"Z01,A01","1,1","Conducteur,Conducteur","Indemne,Blessé","Masculin,Masculin","1992,1960","Domicile – travail,Domicile – travail","Ceinture,Casque","Oui,Oui",None,"Se déplaçant,Se déplaçant",None


In [158]:
#supprimer les colonnes avec missing values supérieures à 50%
col_a_supprimer = ['localisation_du_pieton', 'pieton_seul_ou_non']
df_usagers = supprimer_colonnes(df_usagers, col_a_supprimer)

In [159]:
#Harmonisation de la colonne 'sexe'
# prend les 2 premiers caractères
df_usagers['sexe'] = df_usagers['sexe'].astype(str).str[:2].str.lower().str.replace(' ', '')  
df_usagers.groupby('sexe').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
mapping_sexe = {
    'ma': 'homme',
    'fé': 'femme'
}
harmonise_colonne_df(df_usagers,'sexe',mapping_sexe)
print('\napres harmonisation:')
print(df_usagers.groupby('sexe').size().reset_index(name='counts').sort_values(by='counts', ascending=False))


apres harmonisation:
    sexe  counts
1  homme  332112
0  femme  143799


In [164]:
df_usagers.head(1)

,identifiant_de_l_accident,identifiant_vehicule,place,categorie_d_usager,gravite,sexe,annee_de_naissance,motif_trajet,existence_equipement_de_securite,utilisation_equipement_de_securite,action_pieton
0,201700033619,"B01,A01","1,1,2,2","Conducteur,Conducteur,Passager,Passager","Blessé,Blessé,Blessé,Blessé",homme,"1996,1939,1996,1931","Promenade – loisirs,Promenade – loisirs,Promen...","Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui","Se déplaçant,Se déplaçant,Se déplaçant,Se dépl..."


In [170]:
# df_usagers.groupby('categorie_d_usager').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('gravite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('gravite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('motif_trajet').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('motif_trajet').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('existence_equipement_de_securite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('utilisation_equipement_de_securite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
df_usagers.groupby('action_pieton').size().reset_index(name='counts').sort_values(by='counts', ascending=False)


,action_pieton,counts
520,"Se déplaçant,Se déplaçant",182556
312,Se déplaçant,78113
617,"Se déplaçant,Se déplaçant,Se déplaçant",58645
792,"Se déplaçant,Traversant",25868
847,"Traversant,Se déplaçant",23818
...,...,...
889,"Traversant,Traversant,Se déplaçant,-1",1
890,"Traversant,Traversant,Se déplaçant,Autre",1
891,"Traversant,Traversant,Se déplaçant,Autre,Trave...",1
38,"-1,-1,-1,Se déplaçant,Se déplaçant,-1",1


INFO:__main__:Exécution du script SQL...


INFO:__main__:Script SQL exécuté avec succès
INFO:__main__:Connexion fermée et engine libéré.
